In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
def evaluate(X, X_imputed, count_missing=1):
    mse = (math.sqrt(((X - X_imputed) ** 2).sum(axis=None) / count_missing))
    print(mse)
    return mse

In [9]:
X = np.genfromtxt('../data/completeCasesBoxCox.csv', delimiter=',', skip_header=0)[:, 1:32]
print(X.shape)

(100001, 31)


In [4]:
print(X[:4])

[[ 0.          0.56767846  0.51154428  1.76825943 -0.20340316  0.18133342
  -0.80230814 -0.84099932 -1.07173915  0.31721961  2.18086245  0.03543689
   1.73329086  0.54644128  1.32869664 -0.24583215  1.69688297 -0.7126464
  -1.46663502  0.01120756  1.86993234  1.2671844   1.15676611  1.07871631
  -1.85240026  1.43007004  1.57872588 -0.26870799 -0.29845134  1.16287242
  -0.7081725 ]
 [ 0.          1.31006366  0.98498431  1.14174886  0.06573364  0.64806392
   1.34843632 -1.55841539  0.24131063  0.51819202  0.79822354  0.27580311
  -0.2159915  -0.12295998 -0.18789372  1.32206941 -0.41446232 -0.7126464
   0.78042177 -0.88973542  0.31522822  0.12400729  0.1129711  -0.61081655
  -0.48455026 -1.23684189 -0.58576342  0.06151486 -0.15413855  1.03292294
  -0.13247761]
 [ 1.          0.48459373 -1.25079483 -0.15782387  0.37679223  0.79860055
   4.39962292 -1.06345998  3.30864405  0.70057647  0.79822354  1.22504309
  -0.99925591 -1.04231545  1.43902711  0.17452068 -0.85702726  0.26453557
   0.11707

In [ ]:
MCAR_dir = '/media/brett/Drive/imputation_results/MCAR/'
#MCAR_dir = '../output/sweeps/MCAR/'
for filename in os.listdir(MCAR_dir):
    if not filename.startswith('.'):
        if not filename.endswith('.csv.gz'):
            print(filename)
            X_imputed = pd.read_csv(MCAR_dir + filename, header=None).as_matrix()[:10000, :]
            print(X_imputed.shape)
            print(X_imputed[:4])
            raise Exception('stop')

In [ ]:
MCAR_dir = '/media/brett/Drive/imputation_results/MCAR/'
#MCAR_dir = '../output/sweeps/MCAR/'

for filename in os.listdir(MCAR_dir):
    print(filename)
    if not filename.startswith('.'):
        if filename.endswith('.csv.gz'):
            X_imputed = pd.read_csv(MCAR_dir + filename, header=1).as_matrix()[:, 1:]
        else:
            X_imputed = pd.read_csv(MCAR_dir + filename).as_matrix()
        print(X_imputed.shape)
        print(X_imputed[:4])
    raise Exception('stop')

In [ ]:
MCAR_dir = '/media/brett/Drive/imputation_results/MCAR/'
#MCAR_dir = '../output/sweeps/MCAR/'

mcar_scores = pd.DataFrame(columns=['Method', 'Percent Missing', 'Trial', 'MSE'])

i = 0
for filename in os.listdir(MCAR_dir):
    if not filename.startswith('.'):
        missing = filename.split('_')[1]
        trial = filename.split('_')[2]
        
        print(filename)
        if (filename.split('_')[3] == 'r'):
            method = filename.split('_')[3] + '_' + filename.split('_')[4].split('.csv')[0]
        else:
            method = filename.split('_', 3)[3].split('.csv')[0]
           
        if filename.endswith('.csv.gz'):
            X_imputed = pd.read_csv(MCAR_dir + filename).as_matrix()[:10000, 1:]
        else:
            X_imputed = pd.read_csv(MCAR_dir + filename, header=None).as_matrix()

        
        original = pd.read_csv('../data/spikeincsv/MCAR/' + 'MCAR_' + missing + '_' + trial + '.csv',
                               header=None).as_matrix()   
        missing_vals = np.count_nonzero(np.isnan(original))
        score = evaluate(X, X_imputed, count_missing=missing_vals)
    
        i += 1
        print(i, method, missing, trial, score)
        
        mcar_scores.loc[i] = [method, missing, trial, score]
mcar_scores.to_csv('../output/imputation_scores/mcar.csv')

In [ ]:
mar_dir = '/media/brett/Drive/imputation_results/MAR/'
#mar_dir = '../output/sweeps/MAR/'

mar_scores = pd.DataFrame(columns=['Method', 'FeatureA', 'FeatureB', 'Quartile', 'MSE'])

i = 0
for filename in os.listdir(mar_dir):
    if not filename.startswith('.'):
        feature1 = filename.split('_')[1]
        feature2 = filename.split('_')[2]
        quartile = filename.split('_')[3]
    
        if (filename.split('_')[4] == 'r'):
            method = filename.split('_')[4] + '_' + filename.split('_')[5].split('.csv')[0]
        else:
            method = filename.split('_', 4)[4].split('.csv')[0]
            
        if filename.endswith('.csv.gz'):
            X_imputed = pd.read_csv(mar_dir + filename).as_matrix()[:10000, 1:]
        else:
            X_imputed = pd.read_csv(mar_dir + filename, header=None).as_matrix()
       
#         print(filename)
#         print(os.listdir('../data/spikeincsv/MAR'))
#         print('../data/spikeincsv/MAR_' + feature1 + '_' + feature2 + '_' + quartile + '.csv')
        original = pd.read_csv('../data/spikeincsv/MAR/' + 'MAR_' + feature1 + '_' + feature2 + '_' + quartile + '.csv',
                               header=None).as_matrix()   
        missing_vals = np.count_nonzero(np.isnan(original))
        score = evaluate(X, X_imputed, count_missing=missing_vals)
        
        print(i, method, feature1, feature2, quartile, score)
        mar_scores.loc[i] = [method, feature1, feature2, quartile, score]
        i += 1

mar_scores.to_csv('../output/imputation_scores/mar.csv')
        

In [ ]:
mnar_dir = '/media/brett/Drive/imputation_results/MNAR/'
#mnar_dir = '../output/sweeps/MNAR/'

mnar_scores = pd.DataFrame(columns=['Method', 'Feature', 'Quartile', 'MSE'])

i = 0
for filename in os.listdir(mnar_dir):
    if not filename.startswith('.'):
        feature = filename.split('_')[1]
        quartile = filename.split('_')[2]
            
        if (filename.split('_')[3] == 'r'):
            method = filename.split('_')[3] + '_' + filename.split('_')[4].split('.csv')[0]
        else:
            method = filename.split('_', 3)[3].split('.csv')[0]
            
        if filename.endswith('.csv.gz'):
            X_imputed = pd.read_csv(mnar_dir + filename).as_matrix()[:10000, 1:]
        else:
            X_imputed = pd.read_csv(mnar_dir + filename, header=None).as_matrix()
       
        original = pd.read_csv('../data/spikeincsv/MNAR/' + 'MNAR_' + feature + '_' + quartile + '.csv',
                               header=None).as_matrix()   
        missing_vals = np.count_nonzero(np.isnan(original))
        score = evaluate(X, X_imputed, count_missing=missing_vals)
        
        print(i, method, feature, quartile, score)
        mnar_scores.loc[i] = [method, feature, quartile, score]
        i += 1

mnar_scores.to_csv('../output/imputation_scores/mnar.csv')
        

In [11]:
match_dir = '../output/sweeps/MATCH/'

match_scores = pd.DataFrame(columns=['Method', 'MSE'])

i = 0
for filename in os.listdir(match_dir):
    if not filename.startswith('.'):
        if len(filename.split('_')) > 3:
            if (filename.split('_')[3] == 'r'):
                method = filename.split('_')[3] + '_' + filename.split('_')[4].split('.csv')[0]
            else:
                method = filename.split('_', 2)[2].split('.csv')[0]
        else:
            method = filename.split('_', 2)[2].split('.csv')[0]
#         print(method)
        if filename.endswith('.csv.gz'):
            X_imputed = pd.read_csv(match_dir + filename).as_matrix()[:9999, 1:]
        else:
            X_imputed = pd.read_csv(match_dir + filename).as_matrix()[:9999, :]
        
#         print(filename)
        original = pd.read_csv('../data/spikeincsv/MATCH/part_' + filename.split('_')[1] +
                               '.csv').as_matrix()[:9999, 2:].astype('float32')   
        
        print(X_imputed.shape, original.shape)
        part = int(filename.split('_')[1])
        if part == 1:
            X_section = X[((part-1) * 10000 + 1):(part*10000)]
        else:
            X_section = X[((part-1) * 10000 + 2):(part*10000+1)]         

        original = pd.read_csv('../data/spikeincsv/MATCH/part_' + filename.split('_')[1] +
                               '.csv').as_matrix()[:9999, 2:].astype('float32')  

        print(X_section.shape)
        missing_vals = np.count_nonzero(np.isnan(original))
#         print(missing_vals)
        score = evaluate(X_section, X_imputed, count_missing=missing_vals)
        
        print(i, method, score)
        match_scores.loc[i] = [method, score]
        i += 1
    
#         if i > 5:
#             raise Exception('stop')

match_scores.to_csv('../output/imputation_scores/match.csv')

((9999, 31), (9999, 31))
(9999, 31)
0.976931530762
(0, 'si_s_half', 0.9769315307617046)
((9999, 31), (9999, 31))
(9999, 31)
1.42331270984
(1, 'r_sample', 1.4233127098398093)
((9999, 31), (9999, 31))
(9999, 31)
0.974591036238
(2, 'si_s_half', 0.9745910362376258)
((9999, 31), (9999, 31))
(9999, 31)
0.943313504999
(3, 'mice_col_lambda_reg_01', 0.9433135049988305)
((9999, 31), (9999, 31))
(9999, 31)
0.958991624942
(4, 'mice_col_lambda_reg_25', 0.9589916249415914)
((9999, 31), (9999, 31))
(9999, 31)
1.6068136023
(5, 'svd_12', 1.6068136022977146)
((9999, 31), (9999, 31))
(9999, 31)
1.00716107314
(6, 'mice_pmm_X', 1.0071610731369165)
((9999, 31), (9999, 31))
(9999, 31)
1.62692028723
(7, 'svd_16', 1.626920287226011)
((9999, 31), (9999, 31))
(9999, 31)
0.969256666501
(8, 'knn_6000', 0.9692566665013026)
((9999, 31), (9999, 31))
(9999, 31)
0.953947551098
(9, 'knn_243', 0.953947551098251)
((9999, 31), (9999, 31))
(9999, 31)
1.22019498006
(10, 'mice_pmm_lambda_reg_25', 1.220194980059317)
((9999, 31

In [10]:
match_dir = '../output/sweeps/MATCH/'
match_per_var_scores = pd.DataFrame(columns=['Method', 'Var_id', 'MSE'])
i=0

for filename in os.listdir(match_dir):
    if filename.startswith('part_1_'):
        if len(filename.split('_')) > 3:
            if (filename.split('_')[3] == 'r'):
                method = filename.split('_')[3] + '_' + filename.split('_')[4].split('.csv')[0]
            else:
                method = filename.split('_', 2)[2].split('.csv')[0]
        else:
            method = filename.split('_', 2)[2].split('.csv')[0]
        print(method)
        if filename.endswith('.csv.gz'):
            X_imputed = pd.read_csv(match_dir + filename).as_matrix()[:9999, 1:]
        else:
            X_imputed = pd.read_csv(match_dir + filename).as_matrix()[:9999, :]
        
        part = int(filename.split('_')[1])
        if part == 1:
            X_section = X[((part-1) * 10000 + 1):(part*10000)]
        else:
            X_section = X[((part-1) * 10000 + 2):(part*10000+1)]         

        original = pd.read_csv('../data/spikeincsv/MATCH/part_' + filename.split('_')[1] +
                               '.csv').as_matrix()[:9999, 2:].astype('float32')  
        
        for j in range(original.shape[1]):
            if j > 2:
                missing_vals = np.count_nonzero(np.isnan(original.T[j]))
                score = evaluate(X_section.T[j], X_imputed.T[j], count_missing=missing_vals)
                print('Missing: ', missing_vals)
                print('Score: ', score)
                
                match_per_var_scores.loc[i] = [method, j, score]
                i += 1
#                 if i>100:
#                     print(match_per_var_scores)
#                     raise Exception('stop')
match_per_var_scores.to_csv('../output/imputation_scores/match_by_var.csv')

mice_col_lambda_reg_01
0.871305290577
('Missing: ', 1630)
('Score: ', 0.8713052905767774)
0.95896928201
('Missing: ', 1692)
('Score: ', 0.9589692820102438)
0.968347875915
('Missing: ', 1711)
('Score: ', 0.9683478759151672)
1.021786416
('Missing: ', 1732)
('Score: ', 1.0217864159972079)
0.964816732708
('Missing: ', 1732)
('Score: ', 0.9648167327081785)
1.04249911096
('Missing: ', 1732)
('Score: ', 1.0424991109559605)
0.869862746636
('Missing: ', 1716)
('Score: ', 0.8698627466358076)
1.0028272247
('Missing: ', 1787)
('Score: ', 1.0028272246991468)
0.986918800564
('Missing: ', 1840)
('Score: ', 0.9869188005643762)
0.956184240678
('Missing: ', 1857)
('Score: ', 0.9561842406784188)
0.907197633945
('Missing: ', 1865)
('Score: ', 0.9071976339445704)
0.936929930511
('Missing: ', 1864)
('Score: ', 0.9369299305105767)
0.863389116524
('Missing: ', 1832)
('Score: ', 0.8633891165239261)
0.914856560864
('Missing: ', 1918)
('Score: ', 0.9148565608636867)
0.935256673762
('Missing: ', 1903)
('Score: ',